In [1]:
import tkinter as tk
from tkinter import ttk
import xml.etree.ElementTree as ET
import os
import subprocess

# Define master folder and subfolder paths
master_folder = "C:\\03 MyProject\\AutomationSequences"
subfolder_paths = {
    "UL": os.path.join(master_folder, "UL"),
    "AH": os.path.join(master_folder, "AH"),
    "SPIA": os.path.join(master_folder, "SPIA")
}

# Define PowerShell script paths for each sequence
powershell_scripts = {
    "UL": os.path.join(subfolder_paths["UL"], "GetusernameUL.ps1"),
    "AH": os.path.join(subfolder_paths["AH"], "GetusernameAH.ps1"),
    "SPIA": os.path.join(subfolder_paths["SPIA"], "GetusernameSPIA.ps1")
}

def get_automation_folders(master_folder):
    try:
        return [f for f in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, f))]
    except FileNotFoundError:
        print(f"Error: Master folder '{master_folder}' not found.")
        return []

def get_xml_file_path(automation_folder):
    cfg_folder = os.path.join(automation_folder, "CFG")
    if not os.path.exists(cfg_folder):
        os.makedirs(cfg_folder)  # Create "CFG" folder if it doesn't exist
    for filename in os.listdir(cfg_folder):
        if filename.lower() == "config.xml":
            return os.path.join(cfg_folder, filename)
    return None

def update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value, new_plans_value):
    if os.path.exists(xml_file):
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            # Update StudyRun if exists
            study_run_tag = root.find("StudyRun")
            if study_run_tag is not None:
                study_run_tag.text = study_run_value
            else:
                print("StudyRun tag not found in XML.")

            # Update SequenceStartDate if exists
            sequence_start_date_tag = root.find("SequenceStartDate")
            if sequence_start_date_tag is not None:
                sequence_start_date_tag.text = sequence_start_date_value
            else:
                print("SequenceStartDate tag not found in XML.")

            # Update NewRateFiles if exists
            new_rate_files_tag = root.find("NewRateFiles")
            if new_rate_files_tag is not None:
                new_rate_files_tag.text = new_rate_files_value
            else:
                print("NewRateFiles tag not found in XML.")
                
            # Update NewPlans if exists
            new_plans_tag = root.find("NewPlans")
            if new_plans_tag is not None:
                new_plans_tag.text = new_plans_value
            else:
                print("NewPlans tag not found in XML.")

            tree.write(xml_file)
            print(f"Successfully updated tags in {xml_file}.")
            update_status_label.config(text="XML updated successfully", fg="green")  # Update status label
        except ET.ParseError as e:
            print(f"Error parsing XML file: {e}")
        except Exception as e:
            print(f"Error updating XML file: {e}")
    else:
        print(f"Error: XML file '{xml_file}' not found.")

def display_xml_content(xml_file):
    if xml_file:
        try:
            with open(xml_file, 'r') as f:
                xml_data = f.read()
                text_area.delete("1.0", tk.END)
                text_area.insert(tk.INSERT, xml_data)
                print(f"XML content of {xml_file} displayed in the UI.")
        except Exception as e:
            print(f"Error reading XML file: {e}")
            text_area.delete("1.0", tk.END)
            text_area.insert(tk.INSERT, "Error: Could not display XML content.")
    else:
        print("No XML file found for the selected sequence.")
        text_area.delete("1.0", tk.END)
        text_area.insert(tk.INSERT, "No XML file found.")

def combobox_selected(event):
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = subfolder_paths[selected_sequence]
        xml_file = get_xml_file_path(automation_folder)
        display_xml_content(xml_file)
        script_path_label.config(text=powershell_scripts[selected_sequence])

def save_button_clicked():
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = subfolder_paths[selected_sequence]
        xml_file = get_xml_file_path(automation_folder)

        study_run_value = entry_study_run.get()
        sequence_start_date_value = entry_sequence_start_date.get()
        new_rate_files_value = new_rate_files_combobox.get()
        new_plans_value = new_plans_combobox.get()

        if not study_run_value or not sequence_start_date_value:
            update_status_label.config(text="StudyRun and SequenceStartDate cannot be blank.", fg="red")
            return

        update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value, new_plans_value)
        display_xml_content(xml_file)

def run_powershell_script():
    selected_sequence = combobox.get()
    if selected_sequence:
        script_path = powershell_scripts[selected_sequence]
        try:
            subprocess.run(["powershell", "-File", script_path], check=True)
            print("PowerShell script executed successfully.")
            script_status_label.config(text="PowerShell script executed successfully.", fg="green")
        except subprocess.CalledProcessError as e:
            print(f"Error executing PowerShell script: {e}")
            script_status_label.config(text="Error executing PowerShell script.", fg="red")

def validate_study_run(text):
    return len(text) <= 6

def validate_sequence_start_date(text):
    return len(text) <= 8 and text.isdigit()

def validate_true_false(text):
    return text in ["True", "False", ""]

root = tk.Tk()
root.title("Measure - Experience Study Processing")
root.geometry('700x500')

# Defining Labels
lbl_Header = tk.Label(root, text="Experience Study Processing", font=('Comic Sans MS', 15, 'bold')).pack(pady=10)
lbl_confg = tk.Label(root, text="Select Measure Automation Sequence", font=('Verdana', 10, 'bold')).pack()

# List of Automation Sequences
automation_sequences = ["UL", "AH", "SPIA"]

# Create the combobox
combobox = ttk.Combobox(root, values=automation_sequences, width=35)
combobox.pack()
combobox.bind("<<ComboboxSelected>>", combobox_selected)

# Create a text area to display the XML content
text_area = tk.Text(root, width=70, height=20)
text_area.pack(padx=10, pady=10)

# Create input fields for updating XML tags
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

tk.Label(input_frame, text="StudyRun:").grid(row=0, column=0, padx=5, pady=5)
study_run_var = tk.StringVar()
entry_study_run = tk.Entry(input_frame, textvariable=study_run_var, validate="key", validatecommand=(root.register(validate_study_run), '%P'))
entry_study_run.grid(row=0, column=1, padx=5, pady=5)

tk.Label(input_frame, text="SequenceStartDate:").grid(row=0, column=2, padx=5, pady=5)
sequence_start_date_var = tk.StringVar()
entry_sequence_start_date = tk.Entry(input_frame, textvariable=sequence_start_date_var, validate="key", validatecommand=(root.register(validate_sequence_start_date), '%P'))
entry_sequence_start_date.grid(row=0, column=3, padx=5, pady=5)

tk.Label(input_frame, text="New Rate Files:").grid(row=1, column=0, padx=5, pady=5)
new_rate_files_var = tk.StringVar()
new_rate_files_combobox = ttk.Combobox(input_frame, textvariable=new_rate_files_var, values=["True", "False"], validate="focusout", validatecommand=(root.register(validate_true_false), '%P'))
new_rate_files_combobox.grid(row=1, column=1, padx=5, pady=5)

tk.Label(input_frame, text="New Plans:").grid(row=1, column=2, padx=5, pady=5)
new_plans_var = tk.StringVar()
new_plans_combobox = ttk.Combobox(input_frame, textvariable=new_plans_var, values=["True", "False"], validate="focusout", validatecommand=(root.register(validate_true_false), '%P'))
new_plans_combobox.grid(row=1, column=3, padx=5, pady=5)

# Frame for buttons and labels
button_frame = tk.Frame(root)
button_frame.pack(pady=10)

# Save XML button
save_button = tk.Button(button_frame, text="Save XML", command=save_button_clicked)
save_button.pack(side=tk.LEFT, padx=5)

# Run PowerShell Script button
run_ps_button = tk.Button(button_frame, text="Run PowerShell Script", command=run_powershell_script)
run_ps_button.pack(side=tk.LEFT, padx=5)

# Label to show XML update status
update_status_label = tk.Label(button_frame, text="", font=('Verdana', 8))
update_status_label.pack(side=tk.LEFT, padx=5)

# Label to show script path
script_path_label = tk.Label(button_frame, text="", font=('Verdana', 8))
script_path_label.pack(side=tk.LEFT, padx=5)

# Label to show script execution status
script_status_label = tk.Label(root, text="", font=('Verdana', 8))
script_status_label.pack(pady=5)

root.mainloop()


In [3]:
#Sunday - 20th Aug 2024 - working good
import tkinter as tk
from tkinter import ttk
#import xml.etree.ElementTree as ET
from lxml import etree as ET
import os
import subprocess

# Define master folder and subfolder paths
master_folder = "C:\\03 MyProject\\AutomationSequences"
subfolder_paths = {
    "UL": os.path.join(master_folder, "UL"),
    "AH": os.path.join(master_folder, "AH"),
    "SPIA": os.path.join(master_folder, "SPIA")
}

# Define PowerShell script paths for each sequence
powershell_scripts = {
    "UL": os.path.join(subfolder_paths["UL"], "GetusernameUL.ps1"),
    "AH": os.path.join(subfolder_paths["AH"], "GetusernameAH.ps1"),
    "SPIA": os.path.join(subfolder_paths["SPIA"], "GetusernameSPIA.ps1")
}

def get_automation_folders(master_folder):
    try:
        return [f for f in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, f))]
    except FileNotFoundError:
        print(f"Error: Master folder '{master_folder}' not found.")
        return []

def get_xml_file_path(automation_folder):
    cfg_folder = os.path.join(automation_folder, "CFG")
    if not os.path.exists(cfg_folder):
        os.makedirs(cfg_folder)  # Create "CFG" folder if it doesn't exist
    for filename in os.listdir(cfg_folder):
        if filename.lower() == "config.xml":
            return os.path.join(cfg_folder, filename)
    return None

def update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value, new_plans_value):
    if os.path.exists(xml_file):
        try:
            parser = ET.XMLParser(remove_blank_text=True)
            tree = ET.parse(xml_file, parser)
            root = tree.getroot()

            # Update StudyRun if exists
            study_run_tag = root.find("StudyRun")
            if study_run_tag is not None:
                study_run_tag.text = study_run_value
            else:
                print("StudyRun tag not found in XML.")

            # Update SequenceStartDate if exists
            sequence_start_date_tag = root.find("SequenceStartDate")
            if sequence_start_date_tag is not None:
                sequence_start_date_tag.text = sequence_start_date_value
            else:
                print("SequenceStartDate tag not found in XML.")

            # Update NewRateFiles if exists
            new_rate_files_tag = root.find("NewRateFiles")
            if new_rate_files_tag is not None:
                new_rate_files_tag.text = new_rate_files_value
            else:
                print("NewRateFiles tag not found in XML.")
                
            # Update NewPlans if exists
            new_plans_tag = root.find("NewPlans")
            if new_plans_tag is not None:
                new_plans_tag.text = new_plans_value
            else:
                print("NewPlans tag not found in XML.")

            tree.write(xml_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")
            print(f"Successfully updated tags in {xml_file}.")
            update_status_label.config(text="XML updated successfully", fg="green")  # Update status label
        except ET.XMLSyntaxError as e:
            print(f"Error parsing XML file: {e}")
        except Exception as e:
            print(f"Error updating XML file: {e}")
    else:
        print(f"Error: XML file '{xml_file}' not found.")

def display_xml_content(xml_file):
    if xml_file:
        try:
            with open(xml_file, 'r') as f:
                xml_data = f.read()
                text_area.delete("1.0", tk.END)
                text_area.insert(tk.INSERT, xml_data)
                print(f"XML content of {xml_file} displayed in the UI.")
        except Exception as e:
            print(f"Error reading XML file: {e}")
            text_area.delete("1.0", tk.END)
            text_area.insert(tk.INSERT, "Error: Could not display XML content.")
    else:
        print("No XML file found for the selected sequence.")
        text_area.delete("1.0", tk.END)
        text_area.insert(tk.INSERT, "No XML file found.")

def combobox_selected(event):
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = subfolder_paths[selected_sequence]
        xml_file = get_xml_file_path(automation_folder)
        display_xml_content(xml_file)
        script_path_label.config(text=powershell_scripts[selected_sequence])

def save_button_clicked():
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = subfolder_paths[selected_sequence]
        xml_file = get_xml_file_path(automation_folder)

        study_run_value = entry_study_run.get()
        sequence_start_date_value = entry_sequence_start_date.get()
        new_rate_files_value = new_rate_files_combobox.get()
        new_plans_value = new_plans_combobox.get()

        if not study_run_value or not sequence_start_date_value:
            update_status_label.config(text="StudyRun and SequenceStartDate cannot be blank.", fg="red")
            return

        update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value, new_plans_value)
        display_xml_content(xml_file)

def run_powershell_script():
    selected_sequence = combobox.get()
    if selected_sequence:
        script_path = powershell_scripts[selected_sequence]
        try:
            subprocess.run(["powershell", "-File", script_path], check=True)
            print("PowerShell script executed successfully.")
            script_status_label.config(text="PowerShell script executed successfully.", fg="green")
        except subprocess.CalledProcessError as e:
            print(f"Error executing PowerShell script: {e}")
            script_status_label.config(text="Error executing PowerShell script.", fg="red")
#Restict studyrun text input value
def validate_study_run(text):
    return len(text) <= 6

# Restrict sequence_start_dtae text input value
def validate_sequence_start_date(text):
    return len(text) <= 8 and text.isdigit()

def validate_true_false(text):
    return text in ["True", "False", ""]

root = tk.Tk()
root.title("Measure - Experience Study Processing")
root.geometry('700x600')

# Defining Labels
lbl_Header = tk.Label(root, text="Experience Study Processing", font=('Comic Sans MS', 15, 'bold')).pack(pady=10)
lbl_confg = tk.Label(root, text="Select Measure Automation Sequence", font=('Verdana', 10, 'bold')).pack()

# List of Automation Sequences
automation_sequences = ["UL", "AH", "SPIA"]

# Create the combobox
combobox = ttk.Combobox(root, values=automation_sequences, width=35)
combobox.pack()
combobox.bind("<<ComboboxSelected>>", combobox_selected)

# Create a text area to display the XML content
text_area = tk.Text(root, width=70, height=15)
text_area.pack(padx=10, pady=10)

# Create input fields for updating XML tags
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

tk.Label(input_frame, text="StudyRun:").grid(row=0, column=0, padx=5, pady=5)
study_run_var = tk.StringVar()
entry_study_run = tk.Entry(input_frame, textvariable=study_run_var, validate="key", validatecommand=(root.register(validate_study_run), '%P'))
entry_study_run.grid(row=0, column=1, padx=5, pady=5)

tk.Label(input_frame, text="SequenceStartDate:").grid(row=0, column=2, padx=5, pady=5)
sequence_start_date_var = tk.StringVar()
entry_sequence_start_date = tk.Entry(input_frame, textvariable=sequence_start_date_var, validate="key", validatecommand=(root.register(validate_sequence_start_date), '%P'))
entry_sequence_start_date.grid(row=0, column=3, padx=5, pady=5)

# for SPIA 

#
tk.Label(input_frame, text="New Rate Files:").grid(row=1, column=0, padx=5, pady=5)
new_rate_files_var = tk.StringVar()
new_rate_files_combobox = ttk.Combobox(input_frame, textvariable=new_rate_files_var, values=["True", "False"], validate="focusout", validatecommand=(root.register(validate_true_false), '%P'))
new_rate_files_combobox.grid(row=1, column=1, padx=5, pady=5)

tk.Label(input_frame, text="New Plans:").grid(row=1, column=2, padx=5, pady=5)
new_plans_var = tk.StringVar()
new_plans_combobox = ttk.Combobox(input_frame, textvariable=new_plans_var, values=["True", "False"], validate="focusout", validatecommand=(root.register(validate_true_false), '%P'))
new_plans_combobox.grid(row=1, column=3, padx=5, pady=5)

# Frame for buttons and labels
button_frame = tk.Frame(root)
button_frame.pack(pady=10)

# Save XML button
save_button = tk.Button(button_frame, text="Save XML", command=save_button_clicked)
save_button.grid(row=0, column=0, padx=5, pady=5)

# Run PowerShell Script button
run_script_button = tk.Button(button_frame, text="Run PowerShell Script", command=run_powershell_script)
run_script_button.grid(row=0, column=1, padx=5, pady=5)

# Label to show script path
script_path_label = tk.Label(root, text="", font=('Verdana', 8))
script_path_label.pack(pady=5)

# Label to show script execution status
script_status_label = tk.Label(root, text="", font=('Verdana', 8))
script_status_label.pack(pady=5)

# Label to show XML update status
update_status_label = tk.Label(root, text="", font=('Verdana', 8))
update_status_label.pack(pady=5)

root.mainloop()


XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\AH\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\SPIA\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\AH\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
Successfully updated tags in C:\03 MyProject\AutomationSequences\UL\CFG\config.xml.
XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\AH\CFG\config.xml displayed in the UI.
XML content of C:\03 MyProject\AutomationSequences\UL\CFG\config.xml displayed in the UI.
PowerShell scr